## This code will generate data and store in DBFS

In [0]:
%pip install faker
from faker import Faker
from random import randint, choice
from time import sleep
import pandas as pd
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType

# Initialize the Faker generator
fake = Faker()



# Define the schema for each table
transactions_schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("branch_id", StringType(), True),
    StructField("channel", StringType(), True),
    StructField("transaction_type", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("currency", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("status", StringType(), True)
])

customers_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("address", StringType(), True),
    StructField("credit_score", IntegerType(), True),
    StructField("join_date", TimestampType(), True),
    StructField("last_update", TimestampType(), True)
])

branches_schema = StructType([
    StructField("branch_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("timezone", StringType(), True)
])

# Define the tables and their schemas
tables = {
    "transactions": transactions_schema,
    "customers": customers_schema,
    "branches": branches_schema,
}

# Function to generate data for a specific table
def generate_data(table_name, num_entries=1):
    data = []
    for _ in range(num_entries):
        row = {}
        for column in tables[table_name].fieldNames():
            if column == 'transaction_id':
                row[column] = "T" + str(randint(1000, 9999))
            elif column == 'customer_id':
                row[column] = "C" + str(randint(3000, 8000))
            elif column == 'branch_id':
                row[column] = "B" + str(randint(101, 110)).zfill(3)
            elif column == 'channel':
                row[column] = choice(['online', 'branch', 'mobile', 'ATM'])
            elif column == 'transaction_type':
                row[column] = choice(['deposit', 'withdrawal', 'transfer', 'balance_check'])
            elif column == 'amount':
                row[column] = round(randint(1, 1000) * 100, 2)
            elif column == 'currency':
                row[column] = choice(['USD', 'EUR', 'GBP'])
            elif column == 'timestamp':
                row[column] = fake.date_time_this_year()
            elif column == 'status':
                row[column] = choice(['success', 'pending', 'failed'])
            elif column == 'name':
                if table_name == "branches":
                    row[column] = fake.company_suffix() + " Branch"
                else:
                    row[column] = fake.name()
            elif column == 'email':
                first_name = fake.first_name().lower()
                row[column] = f"{first_name}85@example.net"
            elif column == 'phone':
                row[column] = ''.join([str(randint(0, 9)) for _ in range(10)])
            elif column == 'address':
                row[column] = fake.address()
            elif column == 'credit_score':
                row[column] = randint(300, 850)
            elif column == 'join_date':
                row[column] = fake.date_time_between(start_date='-5y', end_date=datetime.today())
            elif column == 'last_update':
                row[column] = fake.date_time_between(start_date='-1y', end_date=datetime.today())
            elif column == 'location':
                if table_name == "branches":
                    row[column] = choice(['New York, USA', 'London, UK', 'Toronto, Canada', 'Sydney, Australia', 'Los Angeles, USA'])
                else:
                    row[column] = fake.city()
            elif column == 'timezone':
                if table_name == "branches":
                    row[column] = choice(['EST', 'GMT', 'EST', 'AEST', 'PST'])
                else:
                    row[column] = choice(['UTC', 'EST', 'CST', 'MST', 'PST'])
            else:
                row[column] = fake.uuid4()
        data.append(row)
    return pd.DataFrame(data)

# Generate static data for customers and branches
customers_pd_df = generate_data("customers", num_entries=1000)
branches_pd_df = generate_data("branches", num_entries=10)

# Convert to Spark DataFrames with correct schema
customers_spark_df = spark.createDataFrame(customers_pd_df, schema=customers_schema)
branches_spark_df = spark.createDataFrame(branches_pd_df, schema=branches_schema)



# Write the static data to DBFS
customers_spark_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("raw_data.customers")
branches_spark_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("raw_data.branches")

# Function to stream transactions
def stream_data():
    while True:
        # Generate transactions data
        transactions_pd_df = generate_data("transactions", num_entries=5)
        transactions_pd_df['timestamp'] = pd.to_datetime(transactions_pd_df['timestamp'])
        transactions_spark_df = spark.createDataFrame(transactions_pd_df, schema=transactions_schema)
        transactions_spark_df.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("raw_data.transactions")
        print("Generated and saved transactions data")
        sleep(10)  # Adjust the sleep duration as needed

# Start the data streaming
stream_data()


java.io.IOException: Connection failed
	at com.databricks.rpc.Jetty9Client$$anon$1.handleError(Jetty9Client.scala:848)
	at com.databricks.rpc.Jetty9Client$$anon$1.onFailure(Jetty9Client.scala:775)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:197)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:189)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.notifyFailureComplete(HttpExchange.java:275)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.abort(HttpExchange.java:247)
	at shaded.v9_4.org.eclipse.jetty.client.HttpConversation.abort(HttpConversation.java:164)
	at shaded.v9_4.org.eclipse.jetty.client.HttpRequest.abort(HttpRequest.java:821)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.abort(HttpDestination.java:559)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.failed(HttpDestination.java:313)
	at com.databricks.rpc.AbstractConnectionPool$1.failed(AbstractConnectionPool.java:161)
	at shaded.v9_4.org.eclipse.jetty.util.Promise$Wrapper.failed(Promise.java:136)
	at com.databricks.rpc.Jetty9Client$DatabricksHttpDestinationOverHTTP$$anon$2.failed(Jetty9Client.scala:1671)
	at shaded.v9_4.org.eclipse.jetty.util.Promise$Wrapper.failed(Promise.java:136)
	at shaded.v9_4.org.eclipse.jetty.client.HttpClient$1$1.failed(HttpClient.java:660)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport.connectFailed(AbstractConnectorHttpClientTransport.java:138)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport$ClientSelectorManager.connectionFailed(AbstractConnectorHttpClientTransport.java:188)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$Connect.failed(ManagedSelector.java:966)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:369)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1700(ManagedSelector.java:65)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:676)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:535)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:362)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:186)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:173)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:131)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:409)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.$anonfun$run$4(InstrumentedQueuedThreadPool.scala:173)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:279)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:275)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext(AttributionContextTracing.scala:46)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext$(AttributionContextTracing.scala:43)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool.withAttributionContext(InstrumentedQueuedThreadPool.scala:125)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.$anonfun$run$3(InstrumentedQueuedThreadPool.scala:173)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads(QueuedThreadPoolInstrumenter.scala:110)
	at com.databricks.instrumentation.QueuedThreadPoolInstrume